In [1]:
from dotenv import load_dotenv
load_dotenv()
from IPython.display import display, HTML, Markdown

### Summary of the paper

In [3]:
from langchain.docstore.document import Document
from langchain.document_loaders.pdf import PyPDFLoader
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import  LLMChain
from langchain.prompts import PromptTemplate

In [4]:
pdf_path = '/Users/arshath/play/chimiadao/ChimiaResearch/propositional-retrieval/notebooks/pdfs/Augmenting large language models with chemistry tools.pdf'

pdf_loader = PyPDFLoader(pdf_path)
docs = pdf_loader.load()

# Combine all the documents into a single document
text = ""
for doc in docs:
    text += doc.page_content

doc = [Document(
    page_content=text,
    metadata={
        'source': pdf_path.split('/')[-1],
    }
)]

In [5]:
llm = ChatOpenAI(
    model='gpt-4-1106-preview',
    temperature=0.,
)

chain = load_summarize_chain(llm)
summary = chain(doc)

In [6]:
HTML(summary['output_text'])

In [12]:
prompt = """
You are an expert Chemist. You answer questions about chemistry research papers. You are given the context (the research paper) and a question.
You answer the question using the context.
Please do not answer questions that you do not know the answer to.
Output format should be in Markdown.

```context: {context}```
```question: {question}```
```answer:
"""


llm = ChatOpenAI(
    model='gpt-4-1106-preview',
    temperature=0.,
)

chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt),
)

In [13]:
def ask(question):
    return chain.run(
        context=doc,
        question=question,
    )

In [15]:
Markdown(ask('What is the name of the paper?'))

Augmenting large language models with chemistry tools

In [16]:
Markdown(ask('What are the 18 expert designed tools mentioneed in the paper?'))

The 18 expert-designed tools mentioned in the paper are classified into general tools, molecule tools, safety tools, and chemical reaction tools. They are as follows:

### General Tools:
1. **WebSearch**: Uses SerpAPI to query search engines and compile impressions from Google search results.
2. **LitSearch**: Extracts information from scientific documents to provide accurate answers.
3. **Python REPL**: Provides a functional Python shell for running Python code.
4. **Human**: Direct interface for human interaction, allowing the engine to ask questions and expect responses from the user.

### Molecule Tools:
5. **Name2SMILES**: Converts the name or CAS number of a molecule to its SMILES representation.
6. **SMILES2Price**: Provides information on the commercial cost and availability of a specific molecule.
7. **Name2CAS**: Determines the CAS number of a molecule from various identifiers.
8. **Similarity**: Evaluates the similarity between two molecules using the Tanimoto similarity measure.
9. **ModifyMol**: Makes alterations to a given molecule to generate a local chemical space around it.
10. **PatentCheck**: Checks if a molecule has been patented.
11. **FuncGroups**: Identifies functional groups within a molecule.

### Safety Tools:
12. **ControlledChemicalCheck**: Checks against lists of recognized Chemical Weapons and Precursors to ensure safety.
13. **ExplosiveCheck**: Identifies explosive molecules using the GHS system.
14. **SafetySummary**: Provides a general safety overview for any given molecule.

### Chemical Reaction Tools:
15. **NameRXN**: Identifies and classifies chemical reactions based on a database of named reactions.
16. **ReactionPredict**: Predicts chemical reactions using the RXN4Chemistry API.
17. **ReactionPlanner**: Assists in planning a synthetic route for a desired target molecule.
18. **ReactionExecute**: Allows direct interaction with a robotic chemistry lab platform for planning, adapting, and executing syntheses.

These tools are integrated into the ChemCrow platform to enhance its capabilities in automating a diverse set of chemical tasks.

In [17]:
Markdown(ask('please explain the general tools in detail?'))

### General Tools

The general tools described in the research paper are designed to augment the capabilities of large language models (LLMs) like GPT-4, enabling them to perform a variety of tasks that require external information or specific functionalities. Here's a detailed explanation of each general tool:

#### WebSearch
- **Purpose**: To provide the LLM with the ability to access and retrieve information from the web.
- **Functionality**: It uses SerpAPI to perform searches on search engines and compiles impressions from the first page of Google search results.
- **Usage**: It acts as a starting point when the LLM encounters a query it cannot address or is unsure of the appropriate tool to use. It helps the LLM to expand its knowledge base and verify the accuracy of the information it provides.

#### LitSearch
- **Purpose**: To extract relevant information from scientific documents to provide accurate answers to questions.
- **Functionality**: It uses the paper-qa python package, OpenAI Embeddings, and FAISS to embed and search through documents efficiently. A language model generates answers based on embedded vectors.
- **Usage**: It embeds documents and queries into vectors, searches for the top passages, and creates summaries of these passages in relation to the query. These summaries are then used by the LLM to generate informed answers, with references to relevant papers.

#### Python REPL
- **Purpose**: To provide the LLM with a functional Python shell for writing and running Python code.
- **Functionality**: It enables the LLM to perform numerical computations, train AI models, and conduct data analysis directly within the Python environment.
- **Usage**: It is used for tasks that require the execution of Python code, such as data manipulation or running algorithms that are not natively supported by the LLM.

#### Human
- **Purpose**: To serve as a direct interface for human interaction with the LLM.
- **Functionality**: It allows the LLM to ask a question and expect a response from the user.
- **Usage**: It is used when the LLM encounters difficulty or uncertainty and needs human input. It can also be used to give users more control over the LLM's actions by instructing it to request permission for certain tasks.

These general tools are integrated into the LLM through the LangChain framework, which facilitates the development of language model applications and supports the integration of external tools. By using these tools, the LLM can perform tasks that go beyond its native capabilities, such as accessing up-to-date information from the web, processing scientific literature, executing Python code, and interacting with humans for guidance and decision-making.

In [20]:
Markdown(ask('Please explain the chemistry reaction tools in detail?'))

### Chemistry Reaction Tools

ChemCrow integrates several chemistry reaction tools to enhance its capabilities in predicting and planning chemical reactions. Here's a detailed explanation of the reaction tools mentioned in the context:

#### NameRXN
- **Purpose**: To identify and classify chemical reactions based on a database of named reactions.
- **Functionality**: Given a reaction SMILES string, NameRXN returns a classification code and the name of the reaction in natural language.
- **Application**: Helps in understanding reaction mechanisms, selecting catalysts, and optimizing experimental conditions.

#### ReactionPredict
- **Purpose**: To predict the outcome of chemical reactions using a machine learning model.
- **Functionality**: Utilizes the RXN4Chemistry API from IBM Research, which is based on the Molecular Transformer model, to predict the product of a given set of reactants.
- **Application**: Provides accurate chemical reaction predictions that are not readily available from databases, simulating the abstract reasoning of chemists.

#### ReactionPlanner
- **Purpose**: To assist in planning multi-step synthetic routes for desired target molecules.
- **Functionality**: Employs the RXN4Chemistry API to generate a sequence of reactions leading to the target molecule, including conditions, additives, and solvents.
- **Application**: Aids in devising efficient synthetic pathways and translating machine-readable actions into natural language for better understanding.

#### ReactionExecute
- **Purpose**: To enable direct interaction with the physical world through a robotic chemistry lab platform.
- **Functionality**: Plans, adapts, and executes the synthesis of a given molecule using the RXN4Chemistry API. It iteratively adapts the synthesis procedure based on the platform's feedback until the procedure is valid for execution.
- **Application**: Allows for the autonomous execution of standardized synthesis procedures on a robotic platform, reducing human intervention and lab safety concerns.

These tools collectively enhance ChemCrow's ability to autonomously predict and plan chemical syntheses, bridging the gap between computational predictions and experimental execution. They provide the LLM with a deeper understanding of chemistry and the ability to interact with the physical world, making it a powerful assistant for chemists.

In [22]:
Markdown(ask('Please explain the safety tools in detail?'))

The safety tools integrated into ChemCrow are designed to assess the potential risks associated with chemical substances and reactions. These tools help ensure that the recommendations provided by the LLM are not only chemically accurate but also safe for practical implementation. Here's a detailed explanation of each safety tool:

### ControlledChemicalCheck
- **Purpose**: To prevent the recommendation or use of chemicals that are regulated or banned due to their potential use in chemical weapons.
- **Function**: Checks a molecule's CAS number against lists of recognized Chemical Weapons and Precursors, such as those from the Organisation for the Prohibition of Chemical Weapons (Schedules 1-3) and The Australia Group's Export Control List.
- **Action**: If a molecule is found on these lists, the agent immediately stops execution and does not proceed with any synthesis or recommendation involving the controlled substance.

### ExplosiveCheck
- **Purpose**: To identify molecules that are classified as explosives and could pose a significant safety risk if handled improperly.
- **Function**: Queries the PubChem database using molecular identifiers (common name, IUPAC name, or CAS number) to determine if a molecule has a Globally Harmonized System (GHS) rating of "Explosive".
- **Action**: If a molecule is confirmed to be explosive, ChemCrow provides an appropriate warning or error to the user, thereby mitigating the risk associated with handling such substances.

### SafetySummary
- **Purpose**: To provide a comprehensive safety overview of any given molecule, including operational safety, environmental risks, and societal impact.
- **Function**: Gathers safety information from the PubChem database and uses an LLM as an interface to highlight key safety aspects:
  - **Operational Safety**: Potential risks to the operator, such as health concerns when handling the substance.
  - **GHS Information**: General hazards and recommendations for handling the substance.
  - **Environmental Risks**: Concerns about the environmental impact of handling the substance and recommendations for safe disposal.
  - **Societal Impact**: Checks if the substance is a known controlled chemical.
- **Action**: Provides safety guidelines and recommendations for handling the substance. If no information is available, the LLM can fill in the gaps with general safety advice, explicitly stating the lack of specific data.

These safety tools are crucial for responsible chemistry practice, as they help users avoid working with hazardous or illegal substances and ensure adherence to safety protocols.

In [23]:
Markdown(ask('Please explain the molecule tools in detail?'))

The molecule tools described in the research paper are specialized computational utilities designed to perform specific tasks related to molecular structures. These tools are integrated into the ChemCrow platform to enhance its capabilities in handling chemistry-related problems. Here's a detailed explanation of each molecule tool:

### Name2SMILES
- **Purpose**: Converts the name or CAS number of a molecule into its SMILES (Simplified Molecular Input Line Entry System) representation.
- **Functionality**: Users can input a molecule's common name, IUPAC name, or CAS number, and the tool returns the corresponding SMILES string. It primarily queries chem-space and, if unsuccessful, resorts to PubChem and OPSIN for conversion.

### SMILES2Price
- **Purpose**: Provides information on the commercial availability and price of a molecule.
- **Functionality**: Takes a molecule's SMILES string, checks its availability for purchase using molbloom (against the ZINC database), and then retrieves the lowest market price from chem-space API.

### Name2CAS
- **Purpose**: Determines the CAS (Chemical Abstracts Service) number of a molecule.
- **Functionality**: Accepts various molecular identifiers and queries the PubChem database to return the unique CAS number, facilitating precise information retrieval.

### Similarity
- **Purpose**: Evaluates the structural similarity between two molecules.
- **Functionality**: Receives two molecules and calculates the Tanimoto similarity based on ECFP2 molecular fingerprints, aiding in the assessment of molecular analogs.

### ModifyMol
- **Purpose**: Generates modifications to a given molecule to explore similar structures.
- **Functionality**: Uses the SynSpace package to apply medchem reactions and create a local chemical space around the input molecule, returning a modified molecule based on retrosynthesis rules.

### PatentCheck
- **Purpose**: Checks if a molecule has been patented.
- **Functionality**: Uses molbloom to assess compounds against known databases like ZINC, informing the user if a patent exists for the molecule in question.

### FuncGroups
- **Purpose**: Identifies functional groups within a molecule.
- **Functionality**: Analyzes a molecule's SMILES string against a list of named SMARTS patterns to list the functional groups present, crucial for understanding reactivity and properties.

### SMILES2Weight
- **Purpose**: Calculates the molecular weight of a molecule.
- **Functionality**: Takes a SMILES string and uses RDKit to compute the exact molecular weight.

These molecule tools are essential for ChemCrow to autonomously perform tasks such as molecular analysis, similarity assessment, and modification, which are fundamental in various chemical applications like drug discovery and materials science. Integrating these tools with an LLM like GPT-4 allows ChemCrow to handle complex chemistry tasks more effectively by providing accurate molecular information and enabling the exploration of chemical spaces.

In [24]:
Markdown(ask('What LLMs were used and what were the parameters used for each?'))

The LLMs used in the study were GPT-4 and an evaluator LLM, which is also based on GPT-4 but used for evaluation purposes. The parameters used for GPT-4 were a temperature of 0.1. Specific parameters for the evaluator LLM are not detailed in the provided context, but it is mentioned that it was adapted to give a grade based on whether the task is addressed or not, and whether the overall thought process is correct.

In [25]:
Markdown(ask('Could you explain the methods used in this study in great detail?'))

### Methods

#### 5.1 LLMs
Large Language Models (LLMs) have seen rapid advancement and scaling, leading to a variety of applications. The study utilized OpenAI's GPT-4 with a temperature setting of 0.1 to ensure predictable and conservative behavior.

#### 5.2 LLMs application framework – LangChain
LangChain is a framework that supports the development of language model applications. It provides modules for LLMs, prompts, document loaders, chains, indexes, agents, memory, and chat functionality. LangChain was used to integrate external tools with LLMs, allowing for better performance by leveraging the zero-shot reasoning capabilities of LLMs.

#### 5.3 Tools
The study implemented a set of tools categorized into general tools, molecule tools, and chemical reaction tools. These tools are integrated into the LLM to enhance its capabilities in chemistry-related tasks.

##### 5.3.1 General tools
- **WebSearch**: Uses SerpAPI to query search engines and compile information from the web.
- **LitSearch**: Extracts information from scientific documents to provide accurate answers.
- **Python REPL**: Provides a Python shell for the LLM to run Python code.
- **Human**: Allows direct interaction with users for guidance and permission.

##### 5.3.2 Molecule tools
- **Name2SMILES**: Converts molecule names to SMILES strings.
- **SMILES2Price**: Provides information on the commercial cost of molecules.
- **Name2CAS**: Determines the CAS number of molecules.
- **Similarity**: Evaluates the similarity between two molecules using the Tanimoto measure.
- **ModifyMol**: Generates a local chemical space around a molecule using synthesis rules.
- **PatentCheck**: Checks if a molecule has been patented.
- **FuncGroups**: Identifies functional groups within a molecule.

##### 5.3.3 Safety tools
- **ControlledChemicalCheck**: Checks against lists of recognized Chemical Weapons and Precursors.
- **ExplosiveCheck**: Identifies explosive molecules using the GHS.
- **SafetySummary**: Provides a general safety overview for any given molecule.

##### 5.3.4 Chemical reaction tools
- **NameRXN**: Identifies and classifies chemical reactions.
- **ReactionPredict**: Predicts chemical reactions using the RXN4Chemistry API.
- **ReactionPlanner**: Assists in planning synthetic routes for target molecules.
- **ReactionExecute**: Allows interaction with a robotic chemistry lab platform.

#### Data & Code availability
The experiments and an open-source version of the ChemCrow platform, including the main agent setup and a subset of 12 tools, are available on GitHub.

#### Acknowledgements
The study acknowledges support from various institutions and thanks the RXN for Chemistry team for their support and access to the platform for executing the reported syntheses.

### References
The document lists a comprehensive set of references that support the methodologies and tools used in the study.